In [1]:
import pandas as pd
import sys
import os
import sqlite3

DB_PATH = os.path.expanduser("/Users/ultrenzv/Documents/DEV/waiswallet/app/data/waiswallet.db")

conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cursor = conn.cursor()

Checking the Total Income - Dashboard

In [2]:
query = """
SELECT 'Total Income' as Metric,
    PRINTF('₱%,.2f', SUM(amount)) as Value
FROM income_transactions;
"""
    
df = pd.read_sql_query(query, conn)
display(df)

,Metric,Value
0,Total Income,"₱90,000.00"


Checking the Total Expenses - Dashboard

In [3]:
query = """
SELECT SUM(td.line_amount) 
FROM transaction_details td 
WHERE strftime('%Y-%m', td.billing_date) = '2026-02';
"""
    
df = pd.read_sql_query(query, conn)
display(df)

,SUM(td.line_amount)
0,12699.0


Checking the Total Cashback - Dashboard

In [4]:
query = """
SELECT 'Estimated Cashback (YTD)' as Metric,
    PRINTF('₱%,.2f', SUM(cashback_ytd)) as Value
FROM wallets;
"""
    
df = pd.read_sql_query(query, conn)
display(df)

,Metric,Value
0,Estimated Cashback (YTD),₱222.45


Budget vs. Spending (Per Category)

In [5]:
query = """
SELECT c.label as Category,
    PRINTF('₱%,.2f', mb.amount) as Monthly_Budget,
    PRINTF('₱%,.2f', IFNULL(SUM(td.line_amount), 0)) as Amount_Spent,
    ROUND(
        (IFNULL(SUM(td.line_amount), 0) / mb.amount) * 100,
        1
    ) || '%' as Progress
FROM categories c
    JOIN monthly_budgets mb ON c.id = mb.category_id
    LEFT JOIN transaction_details td ON c.id = td.category_id
    AND strftime('%Y-%m', td.billing_date) = '2026-02'
WHERE mb.month_year = '2026-02'
GROUP BY c.id;
"""
    
df = pd.read_sql_query(query, conn)
display(df)

,Category,Monthly_Budget,Amount_Spent,Progress
0,Groceries & Mart,"₱15,000.00","₱3,500.00",23.3%
1,Restaurants & Dining,"₱8,000.00",₱850.00,10.6%
2,Transportation,"₱5,000.00",₱350.00,7.0%
3,Digital Subs,"₱1,500.00",₱549.00,36.6%
4,Shopping,"₱10,000.00","₱1,250.00",12.5%
5,Bills & Utilities,"₱12,000.00","₱6,200.00",51.7%


Lists your cards, their current balances, and cashback performance.

In [6]:
query = """
SELECT name as Card_Name,
    provider as Bank,
    UPPER(type) as Type,
    PRINTF('₱%,.2f', balance) as Current_Balance,
    PRINTF('₱%,.2f', credit_limit) as Credit_Limit,
    PRINTF('₱%,.2f', cashback_ytd) as Cashback_Earned_YTD
FROM wallets;
"""
    
df = pd.read_sql_query(query, conn)
display(df)

,Card_Name,Bank,Type,Current_Balance,Credit_Limit,Cashback_Earned_YTD
0,Amore Cashback,BPI,CREDIT,"₱4,750.00","₱50,000.00",₱152.50
1,EastWest Visa,EastWest,CREDIT,"₱73,399.00","₱100,000.00",₱69.95
2,Main Savings,BDO,DEBIT,"₱181,550.00",₱0.00,₱0.00


Top 5 Recent transactions

In [7]:
query = """
SELECT transaction_date as Date,
    merchant as Merchant,
    PRINTF('₱%,.2f', total_amount) as Amount,
    payment_type as Mode
FROM transaction_headers
ORDER BY transaction_date DESC
LIMIT 5;
"""
    
df = pd.read_sql_query(query, conn)
display(df)

,Date,Merchant,Amount,Mode
0,2026-02-02,GrabFood,₱850.00,NaN
1,2026-02-02,Grab Car,₱350.00,NaN
2,2026-02-02,Shopee,"₱1,250.00",NaN
3,2026-02-02,Meralco,"₱6,200.00",NaN
4,2026-02-02,Beyond the Box,"₱72,000.00",installment


In [2]:
query = """
SELECT *
FROM wallets
"""
    
df = pd.read_sql_query(query, conn)

if df.empty:
    print("ℹ️ Connection successful, but the 'wallets' table is currently empty.")
else:
    display(df)

,id,name,provider,type,balance,available_credit,credit_limit,cycle_day,due_day,monthly_cashback_limit,cashback_ytd,version,created_at,updated_at,benefits
0,1,Amore Cashback,BPI,credit,4750.0,45250.0,50000.0,26.0,15.0,1000.0,152.50,3,2026-02-02 18:20:12,2026-02-07 14:48:48,"{""1"": 4, ""2"": 5}"
1,2,EastWest Visa,EastWest,credit,73399.0,26601.0,100000.0,5.0,25.0,1500.0,69.95,4,2026-02-02 18:20:12,2026-02-02 18:20:12,NaN
2,3,Main Savings,BDO,debit,181550.0,85000.0,NaN,NaN,NaN,0.0,0.00,4,2026-02-02 18:20:12,2026-02-06 18:59:17,NaN


In [9]:
query = """
SELECT 
    c.label AS Category,
    COALESCE(SUM(td.line_amount), 0) AS Total_Spent,
    COALESCE(mb.amount, 0) AS Monthly_Budget,
    COALESCE(mb.amount, 0) - COALESCE(SUM(td.line_amount), 0) AS Remaining_Budget
FROM categories c
LEFT JOIN transaction_details td ON c.id = td.category_id
LEFT JOIN monthly_budgets mb ON c.id = mb.category_id AND mb.month_year = strftime('%Y-%m', 'now')
GROUP BY c.id, c.label, mb.amount
ORDER BY Total_Spent DESC;
"""
    
df = pd.read_sql_query(query, conn)

if df.empty:
    print("ℹ️ Connection successful, but the 'wallets' table is currently empty.")
else:
    display(df)

,Category,Total_Spent,Monthly_Budget,Remaining_Budget
0,Shopping,13250.0,10000.0,-3250.0
1,Bills & Utilities,6200.0,12000.0,5800.0
2,Groceries & Mart,3500.0,15000.0,11500.0
3,Restaurants & Dining,850.0,8000.0,7150.0
4,Digital Subs,549.0,1500.0,951.0
5,Transportation,350.0,5000.0,4650.0


In [ ]:
query = """
SELECT * FROM wallets
"""
    
df = pd.read_sql_query(query, conn)

if df.empty:
    print("ℹ️ Connection successful, but the 'wallets' table is currently empty.")
else:
    display(df)